In [1]:
!pip install torchattacks
!pip install pandas
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
#Librairies for maths
import numpy as np
import matplotlib.pyplot as plt

#Libararies for deep learning
import torch.nn.functional as F
from torchvision import datasets, transforms

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import torch.nn as nn
import torchvision
import numpy as np
from torch.utils.data import DataLoader
import torch.optim as optim

import torchattacks

import math
import time

import random
import pandas as pd
import openpyxl
from torch.optim import Adam,lr_scheduler

In [4]:
import DefaultModel as Util
import GTSRBModel as GTSRBModel
#import Cifar10Model

import Maillard
import CW_L0
import TestUtils

print(dir(Util))
print(dir(GTSRBModel))

['AddSecurity', 'CalculateNorms', 'HardwareIdentification', 'TestLoop', 'Trainloop', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'datasets', 'get_pred', 'imshow', 'np', 'plt', 'torch', 'torchvision', 'transforms']
['Adam', 'Any', 'AugMix', 'Callable', 'ColorJitter', 'Compose', 'DataLoader', 'Dataset', 'GTSRB', 'GTSRB_MODEL', 'GaussianBlur', 'ImportData', 'Optional', 'PIL', 'RandomCrop', 'RandomEqualize', 'RandomHorizontalFlip', 'RandomRotation', 'RandomVerticalFlip', 'Resize', 'ToTensor', 'Tuple', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'csv', 'lr_scheduler', 'nn', 'pathlib', 'random_split', 'torch', 'tqdm', 'train_test_split']


In [5]:
Device = Util.HardwareIdentification()
DimImage = 50
TrainLoader,ValidationLoader, TestLoader = GTSRBModel.ImportData(100, DimImage)

cuda
50
31367
7842
12630


/home/pierre/.local/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [6]:
EPOCHS = 5
LEARNING_RATE = 0.0008
INPUT_DIM = 3*DimImage*DimImage
OUTPUT_DIM = 43
Model = GTSRBModel.GTSRB_MODEL(INPUT_DIM,OUTPUT_DIM, Device).to(Device)

optimizer = torch.optim.Adam(params=Model.parameters(),lr=LEARNING_RATE)
lr_s = lr_scheduler.LinearLR(optimizer,start_factor=1.0,end_factor=0.5,total_iters=10)
loss = nn.CrossEntropyLoss()

In [6]:
%%script false --no-raise-error
Model.compile(train_data=TrainLoader,validation_data=ValidationLoader,epochs=EPOCHS,loss_function=loss,optimizer=optimizer,learning_rate_scheduler=lr_s)
ModelPath = 'ModelSignal.torch'
torch.save(Model.state_dict(), ModelPath)

In [7]:
Model.load_state_dict(torch.load('ModelSignal.torch'))

<All keys matched successfully>

In [8]:
#%%script false --no-raise-error
Util.TestLoop(TestLoader, Model, Device)

Accuracy: 98.94%


In [9]:
%%script false --no-raise-error

Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=1
lr = 0.02

X, y = next(iter(TestLoader))

data = []
start_time = time.time()
data = TestUtils.TestLoopC(Model, 1000, lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "TimeNormal")
TestUtils.SaveData(data, "Best.xlsx")

In [16]:
#%%script false --no-raise-error
from torchvision.utils import save_image
def SaveImages(MyPath, ImagesData, y):
    for i, tensor in enumerate(ImagesData):
        Path = MyPath + "image_" + str(i) + "_" + str(y[i]) + ".png"
        save_image(tensor, Path)

X, y = next(iter(TestLoader))
X = X[20:40]
y = y[20:40]
SaveImages("SignalInitial//", X, y)

In [17]:
#%%script false --no-raise-error

Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=0
lr = 0.2

X, y = next(iter(TestLoader))
X = X[20:40]
y = y[20:40]
TestUtils.SaveLoopC(Model, 10000, lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5, "Signal//")

14 0.7814972 0.22472551 4 5 1000 2 0.2 4 4 5 4
82 4.313662 0.76622254 4 5 1000 2 0.2 33 33 13 33
9 2.1778266 0.77322394 4 5 1000 2 0.2 9 9 9 9
14 1.028712 0.34111995 4 5 1000 2 0.2 3 3 3 3
76 3.264511 0.6089419 4 5 1000 2 0.2 1 1 15 1
20 3.0422235 0.8261127 4 5 1000 2 0.2 11 11 11 11
38 2.3031712 0.51363957 4 5 1000 2 0.2 13 13 13 13
21 3.4320347 0.81447726 4 5 1000 2 0.2 10 10 9 10
12 1.8765165 0.81155974 4 5 1000 2 0.2 9 9 3 9
45 3.0182047 0.82926244 4 5 1000 2 0.2 11 11 30 11
24 1.2211988 0.3726174 4 5 1000 2 0.2 5 5 5 5
30 2.6904347 0.6089946 4 5 1000 2 0.2 17 17 0 17
118 5.7522388 0.73143214 4 5 1000 2 0.2 34 34 35 34
0 0.0 0.0 4 5 1000 2 0.2 23 23 23 1
24 2.5544555 0.7243501 4 5 1000 2 0.2 2 2 1 2
110 6.2527676 0.7946667 4 5 1000 2 0.2 17 17 14 17
12 0.8327023 0.30452934 4 5 1000 2 0.2 3 3 5 3
63 4.9669876 0.90831053 4 5 1000 2 0.2 12 12 17 12
19 2.0503554 0.7422998 4 5 1000 2 0.2 16 16 16 16
0 0.0 0.0 4 5 1000 2 0.2 8 5 5 5


Future exception was never retrieved
future: <Future finished exception=BrokenPipeError(32, 'Broken pipe')>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/unix_events.py", line 676, in write
    n = os.write(self._fileno, data)
BrokenPipeError: [Errno 32] Broken pipe


8 1.4663213 0.6016376 4 5 666.6666666666666 2 0.2 4 4 5 4
81 4.83345 0.71605694 4 5 666.6666666666666 2 0.2 33 33 39 33
6 1.8282279 0.77047503 4 5 666.6666666666666 2 0.2 9 9 9 9
5 1.6557595 0.7939331 4 5 666.6666666666666 2 0.2 3 3 5 3
32 3.032341 0.714484 4 5 666.6666666666666 2 0.2 1 1 1 1
22 3.146058 0.8323408 4 5 666.6666666666666 2 0.2 11 11 11 11
27 2.054221 0.5648731 4 5 666.6666666666666 2 0.2 13 13 13 13
19 2.917812 0.7175689 4 5 666.6666666666666 2 0.2 10 10 18 10
13 1.852679 0.67268354 4 5 666.6666666666666 2 0.2 9 9 3 9
53 2.907548 0.824696 4 5 666.6666666666666 2 0.2 11 11 30 11
23 1.2128574 0.38041273 4 5 666.6666666666666 2 0.2 5 5 5 5
28 2.538898 0.56579244 4 5 666.6666666666666 2 0.2 17 17 14 17
91 6.0814314 0.7159786 4 5 666.6666666666666 2 0.2 34 34 34 34
0 0.0 0.0 4 5 666.6666666666666 2 0.2 23 23 23 1
23 2.5497546 0.7322198 4 5 666.6666666666666 2 0.2 2 2 2 2
91 6.1481643 0.79443026 4 5 666.6666666666666 2 0.2 17 17 14 17
12 0.8426178 0.31075934 4 5 666.6666666666

In [12]:
%%script false --no-raise-error

Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=0
lr = 0.2

X, y = next(iter(TestLoader))

TestedIterations = [10000]

for i in range((len(TestedIterations))):
    data = []
    start_time = time.time()
    data = TestUtils.TestLoopC(Model, TestedIterations[i], lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5)
    end_time = time.time()

    TestUtils.SaveTime(start_time, end_time, "TimeXS" + str(TestedIterations[i]) +".txt")
    TestUtils.SaveData(data, "BestXS" + str(TestedIterations[i]) + ".xlsx")
    

In [13]:
%%script false --no-raise-error

dataJSMA = []

X, y = next(iter(TestLoader))

start_time = time.time()TestUtils.TestJSMA(1,1, Model, X, y, Device)
Data = 
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "TimeJSMAS.txt")
TestUtils.SaveData(Data, "BestJSMAS.xlsx")

In [18]:
from torchvision.utils import save_image
X, y = next(iter(TestLoader))
X = X[20:40]
y = y[20:40]

def SaveJSMA(theta, gamma, Model, X, y, Device, MyPath):
    Data = []
    MyAtk = torchattacks.JSMA(Model, theta=1, gamma=1)
    AdvPFM = MyAtk(X, y)
    for idx in range(len(X)):
        TPred = y[idx:idx+1]
        Ppre = Util.get_pred(Model, X[idx:idx+1], Device)
        Apre = Util.get_pred(Model, AdvPFM[idx:idx+1], Device)
        adv2 = Util.AddSecurity(X[idx:idx+1])
        Spre = Util.get_pred(Model, adv2, Device) 
        Norm = Util.CalculateNorms(TPred, Ppre, Apre, Spre, X[idx:idx+1], AdvPFM[idx:idx+1], 0, 0, 0, 0, 0)
        Path = MyPath + "image_" + str(idx) + "_" + str(Norm[0]) + ".png"
        save_image(AdvPFM[idx:idx+1], Path)
    return [Data]

SaveJSMA(1,1, Model, X, y, Device, "SignalJSMA//")

10 2.561901 0.85882354 0 0 0 0 0 4 4 5 4
150 8.948776 0.9098039 0 0 0 0 0 33 33 4 33
60 6.641938 0.92156863 0 0 0 0 0 9 9 1 9
30 4.9024863 0.94509804 0 0 0 0 0 3 3 5 3
76 2.5265007 0.5921569 0 0 0 0 0 1 1 2 1
156 6.550474 0.88235295 0 0 0 0 0 11 11 11 11
118 6.4086328 0.8666667 0 0 0 0 0 13 13 13 13
66 6.919722 0.96862745 0 0 0 0 0 10 10 1 10
172 8.695086 0.92156863 0 0 0 0 0 9 9 3 9
82 5.7209396 0.8901961 0 0 0 0 0 11 11 2 11
136 8.29458 0.9372549 0 0 0 0 0 5 5 5 5
70 6.137029 0.8901961 0 0 0 0 0 17 17 8 17
244 10.409636 0.94509804 0 0 0 0 0 34 34 5 34
42 6.0432773 0.9764706 0 0 0 0 0 23 23 18 1
60 4.452963 0.80784315 0 0 0 0 0 2 2 3 2
252 10.449661 0.94509804 0 0 0 0 0 17 17 8 17
34 4.0622964 0.80784315 0 0 0 0 0 3 3 1 3
60 7.083011 0.9490196 0 0 0 0 0 12 12 3 12
20 3.1488779 0.89411765 0 0 0 0 0 16 16 16 16
58 6.899046 0.95686275 0 0 0 0 0 8 5 9 16


[[]]

: 

In [15]:
%%script false --no-raise-error
Count=0
X, y = next(iter(TestLoader))
dataCW = []

start_time = time.time()
Data, AX = TestUtils.TestCW(100, 1, 15, Model, X, y, Device, image_size=50, num_channels = 3, DimOutput = OUTPUT_DIM)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "TimeCWS.txt")
TestUtils.SaveData([Data], "BestCWS.xlsx")